In [1]:
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table 


# Fits file

In [2]:

def Sloan(pos , bands = ['u','g','r','i','z'] , size = 20):
    """
    This function simplifies the process of SDSS image retrieving.

    Parameters
    ----------
    pos : astropy.coordinates.sky_coordinate.SkyCoord
        position information generated by astropy. It could be either from name or Ra and Dec.
    bands : list, optional
        SDSS filter selection, by default ['u','g','r','i','z']
    size : int, optional
        The searching cone size, by default 20 [arcsec]
    """
    # Query that region
    xid = SDSS.query_region(pos, spectro=False,radius = size * u.arcsec)

    # a to c process the duplicates.
    a = xid.to_pandas()
    b = a.drop_duplicates(subset=['run','rerun','camcol','field'])
    c = Table([b.ra,b.dec,b.objid,b.run,b.rerun,b.camcol,b.field],names = ('ra','dec','objid','run','rerun','camcol','field'))

    # Start downloading
    for band in bands: # for each band
        im = SDSS.get_images(matches=c, band=band)
        i = 0 # Give the images a counter e.g., SDSS_u_0, SDSS_u_1
        for image in im:
            image.writeto(f'SDSS_{band}_{i}.fits', overwrite=True)
            print(f'{band} image downloaded')
            i += 1


In [3]:
Sloan(SkyCoord.from_name("PTF09gn"))

u image downloaded
g image downloaded
r image downloaded
i image downloaded
z image downloaded


# Radial Search

In [4]:
url = "http://skyserver.sdss.org/dr16/SkyServerWS/SearchTools/RadialSearch?ra=258.2&dec=64&radius=4.1&whichway=equatorial&limit=10&format=json&fp=none&uband=0,17&check_u=u&gband=0,15&check_g=g&whichquery=imaging"

In [5]:
import requests
import pandas as pd

In [6]:
def SDSS_radial(pos, size = .5, format = "json",limit = 10):
    """API provided by SDSS. http://skyserver.sdss.org/dr16/en/help/docs/api.aspx#voservice
    NOTE: The result is not in any order. Do check the coordinates before using the retrieved data.

    Parameters
    ----------
    pos : astropy.coordinates.sky_coordinate.SkyCoord
        position information generated by astropy. It could be either from name or Ra and Dec.
    size : int, optional
        search radius in arcminutes, by default .5
    format : str, optional
        return foramt, by default "json"
        Here I use json, pandas dataframe, html and csv as available inputs. You may type json, pd, html, csv accordingly.
    limit : int, optional
        number of return results, by default 10

    Returns
    -------
    found objects
    """
    
    classification = {1:"cosmic ray", 2:"defect", 3:"galaxy", 4:"ghost", 5:"known object", 6:"star", 7:"star trail", 8:"sky"}

    ra = pos.ra.degree
    dec = pos.dec.degree
    url = f"http://skyserver.sdss.org/dr16/SkyServerWS/SearchTools/RadialSearch?ra={ra}&dec={dec}&radius={size}&whichway=equatorial&limit={limit}&format={format}&whichquery=imaging"
    
    r = requests.get(url)

    if format == "json":
        return r.json()
    elif format == "pd":
        return pd.DataFrame.from_dict(r.json()[0]['Rows']).replace({"type":classification})
    elif format == "html":
        print(f"Visit {url}")
        return True
    elif format == "csv":
        print(f"Visit {url}")
        return r.text
    

In [7]:
SDSS_radial(SkyCoord.from_name("NGC 3424"), format = "pd", limit=10, size = .1)

,objid,run,rerun,camcol,field,obj,type,ra,dec,u,g,r,i,z,Err_u,Err_g,Err_r,Err_i,Err_z
0,1237665016311644292,4550,301,3,47,132,galaxy,162.942903,32.900215,15.32969,13.49522,12.60644,12.1257,11.73268,0.012134,0.002504,0.002998,0.001973,0.002731


In [8]:
def SDSS_Navigator(pos, scale = 0.1, tag = None):
    """Quick access to SDSS DR16 Navigator
    You may check http://skyserver.sdss.org/dr16/en/help/docs/api.aspx#imgcutout for further information

    Parameters
    ----------
    pos : astropy.coordinates.sky_coordinate.SkyCoord
        position information generated by astropy. It could be either from name or Ra and Dec.
    scale : float, optional
        arsec per pixel, by default 0.1
    tag : string, optional
        Effect on image, by default PBS. Chekc the url above for tag information.

    Returns
    -------
    SDSS DR16 Navigator url
    """

    ra = pos.ra.degree
    dec = pos.dec.degree

    if tag == None:
        url = f"http://skyserver.sdss.org/dr16/en/tools/chart/navi.aspx?ra={ra}&dec={dec}&scale={scale}&opt=PBS"
        return url
    else:
        url = f"http://skyserver.sdss.org/dr16/en/tools/chart/navi.aspx?ra={ra}&dec={dec}&scale={scale}&opt={tag}"
        return url

In [9]:
SDSS_Navigator(SkyCoord.from_name("NGC 3424"), scale = 0.5)

'http://skyserver.sdss.org/dr16/en/tools/chart/navi.aspx?ra=162.9430083&dec=32.9006778&scale=0.5&opt=PBS'